In [1]:
# # --- 1. Install Libraries ---
# !pip install sentence-transformers pillow pandas tqdm

# New clean train

In [2]:
# # --- 1. Install Libraries ---
# !pip install sentence-transformers timm pandas tqdm

# Second approach to a clean dataset

In [3]:
# # --- 1. Install Libraries ---
# !pip install sentence-transformers pillow pandas tqdm numpy

In [4]:
# # --- 2. Load Models and Data ---
# import numpy as np
# import torch
# import torch.nn.functional as F
# from sentence_transformers import SentenceTransformer
# from PIL import Image
# from tqdm import tqdm
# import os
# import pandas as pd

# # --- Configuration ---
# # !!! UPDATE THESE PATHS !!!
# # This is the .npz file your main notebook uses
# NPZ_FILE_PATH = "/kaggle/input/train.npz" 
# # This is the path to the images you confirmed
# IMAGES_FOLDER_PATH = "/kaggle/input/train/train/Images"
# # ---------------------

# print("Loading CLIP model...")
# clip_model = SentenceTransformer('clip-ViT-B-32')
# device = clip_model.device
# print(f"CLIP model loaded on device: {device}")

# print("Loading training data from .npz file...")
# try:
#     full_train_data = np.load(NPZ_FILE_PATH, allow_pickle=True)
    
#     # --- Get raw text and image names ---
#     all_texts = full_train_data['captions/text']
#     all_image_names = full_train_data['images/names']
    
#     # This is the 1-to-5 structure
#     num_images = len(all_image_names)
#     num_captions_per_image = len(all_texts) // num_images
    
#     print(f"Loaded {len(all_texts)} captions for {num_images} images.")
    
# except Exception as e:
#     print(f"Error loading data: {e}")
#     print(f"Please check your NPZ_FILE_PATH: {NPZ_FILE_PATH}")

In [5]:
# # --- 3. Score All Pairs ---
# all_pair_scores = [] # This will store (image_index, caption_index, score)
# print(f"Starting to score {num_images} images...")

# for img_idx in tqdm(range(num_images)):
#     try:
#         # --- 1. Get Image Embedding ---
#         img_name = all_image_names[img_idx]
#         img_path = os.path.join(IMAGES_FOLDER_PATH, img_name)
        
#         # Open the image and encode it
#         pil_image = Image.open(img_path)
#         img_emb = clip_model.encode(pil_image, device=device, convert_to_tensor=True)
#         img_emb = F.normalize(img_emb, p=2, dim=0) # Normalize the single vector

#         # --- 2. Get Text Embeddings for this image's 5 captions ---
#         caption_indices = range(img_idx * num_captions_per_image, (img_idx + 1) * num_captions_per_image)
#         captions = [all_texts[cap_idx] for cap_idx in caption_indices]
        
#         # Encode all 5 captions at once
#         text_embs = clip_model.encode(captions, device=device, convert_to_tensor=True)
#         text_embs = F.normalize(text_embs, p=2, dim=1) # Normalize the 5 text vectors

#         # --- 3. Calculate Scores ---
#         # Dot product of 1 img_emb with 5 text_embs
#         sim_scores = (img_emb @ text_embs.T).cpu().numpy()
        
#         # Save each individual score
#         for i, cap_idx in enumerate(caption_indices):
#             all_pair_scores.append((img_idx, cap_idx, float(sim_scores[i])))

#     except Exception as e:
#         print(f"Warning: Failed to process image {img_idx} ({img_name}). Error: {e}")

# print(f"\nFinished scoring. Total (image, caption) pairs scored: {len(all_pair_scores)}")

In [6]:
# # --- 4. Analyze Scores and Filter ---
# import matplotlib.pyplot as plt

# # Convert to a DataFrame for easy analysis
# scores_df = pd.DataFrame(all_pair_scores, columns=['image_index', 'caption_index', 'clip_score'])

# print("Score statistics:")
# print(scores_df['clip_score'].describe())

# # --- Plot a histogram to help pick a threshold ---
# plt.figure(figsize=(10, 4))
# scores_df['clip_score'].hist(bins=100)
# plt.title("Histogram of CLIP Similarity Scores")
# plt.xlabel("CLIP Score")
# plt.ylabel("Frequency")
# plt.vlines(0.28, 0, 1000, colors='r', linestyles='--', label='Example Threshold (0.28)')
# plt.legend()
# plt.show()

# # --- Pick a threshold ---
# # As we discussed, a more aggressive threshold might be better
# SCORE_THRESHOLD = 0.23 # Try 0.28 or 0.25

# # Get all pairs that are *above* the threshold
# clean_pairs_df = scores_df[scores_df['clip_score'] >= SCORE_THRESHOLD]

# # --- Get the indices we want to keep ---
# # This is the list of all individual caption indices (0 to 124,999)
# clean_caption_indices = clean_pairs_df['caption_index'].unique()
# clean_caption_indices.sort()

# # This is the list of all image indices (0 to 24,999)
# # that have AT LEAST ONE good caption
# clean_image_indices = clean_pairs_df['image_index'].unique()
# clean_image_indices.sort()

# print(f"\nOriginal number of captions: {len(all_texts)}")
# print(f"Number of clean captions (score >= {SCORE_THRESHOLD}): {len(clean_caption_indices)}")
# print(f"Total captions removed: {len(all_texts) - len(clean_caption_indices)}")

# print(f"\nOriginal number of images: {num_images}")
# print(f"Number of images with at least 1 clean caption: {len(clean_image_indices)}")

In [7]:
# # --- 5. Save Clean Indices ---

# # Save the numpy array of clean image indices
# np.save("clean_image_indices.npy", clean_image_indices)
# print(f"Saved {len(clean_image_indices)} clean image indices to 'clean_image_indices.npy'")

# # Save the numpy array of clean caption indices
# np.save("clean_caption_indices.npy", clean_caption_indices)
# print(f"Saved {len(clean_caption_indices)} clean caption indices to 'clean_caption_indices.npy'")

# Model and stuff

In [8]:
import random
import os
import torch
import numpy as np

# --- 1. Set a global seed ---
SEED = 42

def seed_everything(seed):
    """Sets all random seeds for reproducibility."""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU
    
    # --- 2. Set CUDNN backend to be deterministic ---
    # This can slow down training, but is necessary for reproducibility
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def seed_worker(worker_id):
    """
    Seeder function for DataLoader workers to ensure reproducible shuffling.
    """
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# --- 3. Run the seeder function ---
seed_everything(SEED)

# --- 4. Create a generator for DataLoaders ---
# This will be passed to your train_loader to ensure reproducible shuffling
g = torch.Generator()
g.manual_seed(SEED)

print(f"All seeds set to {SEED} and CUDNN is in deterministic mode.")

All seeds set to 42 and CUDNN is in deterministic mode.


In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean-dataset/clean_image_indices.npy
/kaggle/input/clean-dataset/clean_caption_indices.npy
/kaggle/input/aml-challenge-dataset/train.npz
/kaggle/input/aml-challenge-dataset/test.clean.npz
/kaggle/input/aml-challenge-dataset/train/train/captions.txt
/kaggle/input/aml-challenge-dataset/train/train/train.npz
/kaggle/input/aml-challenge-dataset/train/train/Images/3463034205.jpg
/kaggle/input/aml-challenge-dataset/train/train/Images/268704620.jpg
/kaggle/input/aml-challenge-dataset/train/train/Images/7535037918.jpg
/kaggle/input/aml-challenge-dataset/train/train/Images/4912369161.jpg
/kaggle/input/aml-challenge-dataset/train/train/Images/6802728196.jpg
/kaggle/input/aml-challenge-dataset/train/train/Images/3346289227.jpg
/kaggle/input/aml-challenge-dataset/train/train/Images/3217056901.jpg
/kaggle/input/aml-challenge-dataset/train/train/Images/272471327.jpg
/kaggle/input/aml-challenge-dataset/train/train/Images/4763916790.jpg
/kaggle/input/aml-challenge-dataset/train/train/Im

In [10]:
# --- Kaggle Data Setup (Updated) ---
import os

# The 'data' directory is no longer needed for images, but we'll keep it in case
# the code saves anything there later. The unzip command is removed.
os.makedirs('data', exist_ok=True)

# Clone the required GitHub repository (needs Internet ON in notebook settings).
!git clone https://github.com/Mamiglia/challenge.git

Cloning into 'challenge'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 98 (delta 39), reused 72 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (98/98), 21.03 MiB | 20.10 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import time


# Image Cleansing

In [12]:
# # --- 2. Data Loading (Modified for K-Fold AND Cleaned Data) ---
# try:
#     full_train_data = np.load("/kaggle/input/train.npz", allow_pickle=True)
#     all_text_embeddings = torch.from_numpy(full_train_data['captions/embeddings']).float()
#     all_image_embeddings = torch.from_numpy(full_train_data['images/embeddings']).float()
#     all_image_ids = full_train_data['images/names'] # From your previous fix

#     num_images_total = all_image_embeddings.shape[0]
#     num_captions_per_image = all_text_embeddings.shape[0] // num_images_total

#     # --- NEW: Load the clean indices ---
#     print("Loading clean image indices...")
#     clean_image_indices = np.load("clean_image_indices.npy") 
#     print(f"Loaded {len(clean_image_indices)} clean image indices.")
    
#     # --- NEW: Filter all data based on these indices ---
#     all_image_embeddings = all_image_embeddings[clean_image_indices]
#     all_image_ids = all_image_ids[clean_image_indices]
    
#     # Re-build text embeddings from only the clean images
#     clean_caption_indices = []
#     for img_idx in clean_image_indices:
#         start = img_idx * num_captions_per_image
#         end = start + num_captions_per_image
#         clean_caption_indices.extend(range(start, end))
        
#     all_text_embeddings = all_text_embeddings[clean_caption_indices]
    
#     # --- Now, proceed as before but with the smaller dataset ---
#     num_images = all_image_embeddings.shape[0]
#     num_captions_per_image = all_text_embeddings.shape[0] // num_images # Should still be 5
    
#     image_indices = np.arange(num_images) # This is now an index for the *clean* array
    
#     kfold = KFold(n_splits=CONFIG['N_FOLDS'], shuffle=True, random_state=CONFIG['RANDOM_STATE'])

#     print(f"CLEAN Data loaded: {all_text_embeddings.shape[0]} text embeddings, {all_image_embeddings.shape[0]} image embeddings.")
#     print(f"KFold splitter created with {CONFIG['N_FOLDS']} splits.")

# except FileNotFoundError:
#     print("Error: Make sure 'train.npz' and 'clean_image_indices.npy' are in the correct directory.")

In [13]:
# --- 1. CONFIGURATION (for MLP-Only Model) ---
import torch

CONFIG = {
    # --- MODIFIED: Single model path template ---
    "MODEL_PATH_TPL": "best_translator_mlp_fold_{}.pth",
    "DEVICE": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    
    # --- Training ---
    "EPOCHS": 100, # Your log shows 100
    "PATIENCE": 25,
    
    # --- K-Fold Configuration ---
    "N_FOLDS": 10,  # Your log shows 5
    "RANDOM_STATE": 42,
    
    # --- Architecture/Loss Params (Using your tuned values) ---
    "HIDDEN_FEATURES": 3072, 
    "NUM_BLOCKS": 2,
    "DROPOUT_RATE": 0.3832296399028748,     
    "LR": 1.7749224152998916e-05,             
    "WEIGHT_DECAY": 0.0008723639231133962,
    
    # --- Hybrid Loss Params ---
    "MARGIN": 0.3898481154806984,
    "TEMPERATURE": 0.010879304099273002,
    "HYBRID_ALPHA": 0.0, # Kept at 0
    "MIXUP_ALPHA": 0.23197266859653176,
    "LABEL_SMOOTHING": 0.12,
    
    # --- General Settings ---
    "BATCH_SIZE": 256,
    "ACCUMULATION_STEPS": 2,
}

In [14]:
# --- 10. Data Loading (MODIFIED for Cleaned Data) ---
from sklearn.model_selection import KFold
import numpy as np
import torch

try:
    # --- 1. Load Full (Noisy) Data ---
    full_train_data = np.load("/kaggle/input/aml-challenge-dataset/train.npz", allow_pickle=True)
    all_text_embeddings_full = torch.from_numpy(full_train_data['captions/embeddings']).float()
    all_image_embeddings_full = torch.from_numpy(full_train_data['images/embeddings']).float()
    all_image_ids_full = full_train_data['images/names']
    
    num_images_total = all_image_embeddings_full.shape[0]
    num_captions_per_image = all_text_embeddings_full.shape[0] // num_images_total

    # --- 2. Load the Clean Indices ---
    print("Loading clean indices from '/kaggle/input/clean-dataset/'...")
    clean_caption_indices = np.load("/kaggle/input/clean-dataset/clean_caption_indices.npy")
    # clean_image_indices = np.load("/kaggle/input/clean-dataset/clean_image_indices.npy")
    print(f"Loaded {len(clean_caption_indices)} clean caption indices.")
    
    # --- 3. Filter Embeddings ---
    # We will build our new dataset from the clean CAPTION indices
    # This creates a 1-to-1 (text, image) dataset
    
    print("Filtering embeddings based on clean indices...")
    # Filter text embeddings (e.g., 100,000 x 1024)
    all_text_embeddings = all_text_embeddings_full[clean_caption_indices]
    
    # Find the corresponding image for each clean caption
    # e.g., caption 12 -> image 2 (12 // 5)
    # e.g., caption 14 -> image 2 (14 // 5)
    corresponding_image_indices = clean_caption_indices // num_captions_per_image
    
    # Filter image embeddings (e.g., 100,000 x 1536)
    all_image_embeddings = all_image_embeddings_full[corresponding_image_indices]
    
    print(f"Clean data shape (text): {all_text_embeddings.shape}")
    print(f"Clean data shape (image): {all_image_embeddings.shape}")

    # --- 4. Setup K-Fold for 1-to-1 Pairs ---
    # We now split the PAIRS, not the images
    num_clean_pairs = len(all_text_embeddings)
    pair_indices = np.arange(num_clean_pairs)
    
    kfold = KFold(n_splits=CONFIG['N_FOLDS'], shuffle=True, random_state=CONFIG['RANDOM_STATE'])

    print(f"\nCLEAN Data loaded: {num_clean_pairs} text/image pairs.")
    print(f"KFold splitter created for {num_clean_pairs} pairs.")

except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Make sure 'train.npz' is at '/kaggle/input/train/train/train.npz'")
    print("And your 'clean-dataset' is attached.")

Loading clean indices from '/kaggle/input/clean-dataset/'...
Loaded 123460 clean caption indices.
Filtering embeddings based on clean indices...
Clean data shape (text): torch.Size([123460, 1024])
Clean data shape (image): torch.Size([123460, 1536])

CLEAN Data loaded: 123460 text/image pairs.
KFold splitter created for 123460 pairs.


# Config and Parameter tuning


# Data Loading

In [15]:
# from sklearn.model_selection import KFold, train_test_split
# import numpy as np
# import torch

# try:
#     # --- FIX 1: Corrected file path ---
#     full_train_data = np.load("/kaggle/input/train.npz", allow_pickle=True) 
    
#     all_text_embeddings = torch.from_numpy(full_train_data['captions/embeddings']).float()
#     all_image_embeddings = torch.from_numpy(full_train_data['images/embeddings']).float()
    
#     # --- FIX 2: Corrected key from 'images/ids' to 'images/names' ---
#     all_image_ids = full_train_data['images/names'] 
    
#     num_images = all_image_embeddings.shape[0]
#     num_captions_per_image = all_text_embeddings.shape[0] // num_images

#     image_indices = np.arange(num_images)
    
#     # Use N_FOLDS from your CONFIG
#     kfold = KFold(n_splits=CONFIG['N_FOLDS'], shuffle=True, random_state=CONFIG['RANDOM_STATE'])

#     print(f"Data loaded: {all_text_embeddings.shape[0]} text embeddings, {all_image_embeddings.shape[0]} image embeddings.")
#     print(f"Image IDs loaded: {len(all_image_ids)} IDs.")
#     print(f"KFold splitter created with {CONFIG['N_FOLDS']} splits.")

# except FileNotFoundError:
#     print("ERROR: Please make sure the 'train.npz' file is in the correct directory.")
#     print("Available files are:")
#     !ls -l /kaggle/input/
# except KeyError:
#     print("ERROR: A key was not found in train.npz.")
#     print("Available keys in the file are:")
#     print(full_train_data.files)

In [16]:
print(full_train_data.files)

['metadata/num_captions', 'metadata/num_images', 'metadata/embedding_dim_text', 'metadata/embedding_dim_image', 'captions/ids', 'captions/text', 'captions/embeddings', 'captions/label', 'images/names', 'images/embeddings']


# Model Architecture

In [17]:
# --- 3. Model Architecture (MODIFIED: MLP-Only) ---
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- 3A. The GeGLU Activation Module (Unchanged) ---
class GeGLU(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.proj = nn.Linear(in_features, out_features * 2)
        self.gelu = nn.GELU()
    def forward(self, x):
        proj_out = self.proj(x)
        a, b = proj_out.chunk(2, dim=-1)
        return self.gelu(a) * b

# --- 3B. The Residual Block with GeGLU (Unchanged) ---
class ResidualBlockGeGLU(nn.Module):
    def __init__(self, features, dropout_rate):
        super().__init__()
        self.block = nn.Sequential(
            GeGLU(features, features), 
            nn.BatchNorm1d(features),
            nn.Dropout(dropout_rate)
        )
    def forward(self, x):
        return x + self.block(x)

# --- 3C. REMOVED TextVariationalEncoder ---
# This class is no longer needed for this architecture.

# --- 3D. The Translator (Your HybridMLP) (Unchanged) ---
# This class is already flexible. 
# 'in_features' will be 1024 (text dim) when we create the model.
class TranslatorMLP(nn.Module):
    def __init__(self, in_features, out_features, hidden_features, num_blocks, dropout_rate):
        super().__init__()
        
        backbone_layers = [
            nn.Linear(in_features, hidden_features, bias=False),
            nn.BatchNorm1d(hidden_features),
            nn.GELU(),
            nn.Dropout(0.5)
        ]
        for _ in range(num_blocks):
            backbone_layers.append(
                ResidualBlockGeGLU(hidden_features, dropout_rate) # Using GeGLU
            )
        self.backbone = nn.Sequential(*backbone_layers)
        
        self.translator_head = nn.Sequential(
            nn.Linear(hidden_features, hidden_features // 2),
            nn.GELU(),
            nn.Linear(hidden_features // 2, out_features) # out_features = 1536
        )

    def forward(self, x):
        shared_representation = self.backbone(x)
        output = self.translator_head(shared_representation)
        # We keep the normalization, as discussed
        return F.normalize(output, p=2, dim=1)

# Helper function

# Loss Function

# Validation functions

# Helper Functions

In [18]:
# --- 5. Helper Functions (for MLP-Only Model) ---
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# --- 5A. Loss Functions (Unchanged) ---
class PureContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.07, label_smoothing=0.12):
        super().__init__()
        self.temperature = temperature
        self.label_smoothing = label_smoothing # <-- MODIFIED
    def forward(self, pred_norm, target_norm):
        sim_matrix = torch.matmul(pred_norm, target_norm.T) / self.temperature
        labels = torch.arange(pred_norm.size(0), device=pred_norm.device)
        return F.cross_entropy(sim_matrix, labels, label_smoothing=self.label_smoothing)
class TripletLoss(nn.Module):
    def __init__(self, margin=0.2):
        super().__init__()
        self.margin = margin
    def forward(self, text_embeds, img_embeds_norm):
        sim_matrix = text_embeds @ img_embeds_norm.T
        positive_scores = torch.diag(sim_matrix)
        mask = torch.eye(text_embeds.size(0), dtype=torch.bool, device=text_embeds.device)
        sim_matrix_masked = sim_matrix.masked_fill(mask, -float('inf'))
        hard_negative_scores = sim_matrix_masked.max(dim=1)[0]
        return F.relu(self.margin - positive_scores + hard_negative_scores).mean()

# --- 5B. FAST VALIDATION (MODIFIED for single model) ---
# This version only takes 3 arguments
def validation_fn(translator_model, val_loader, criterion_triplet, criterion_pure, alpha, device):
    translator_model.eval()
    val_loss = 0
    with torch.no_grad():
        for text_batch, image_batch in val_loader:
            text_batch, image_batch = text_batch.to(device), image_batch.to(device)
            
            pred_embeddings = translator_model(text_batch)
            target_embeddings = F.normalize(image_batch, p=2, dim=1)
            
            loss_triplet = criterion_triplet(pred_embeddings, target_embeddings)
            loss_pure = criterion_pure(pred_embeddings, target_embeddings)
            hybrid_loss = (alpha * loss_triplet) + ((1 - alpha) * loss_pure)
            
            val_loss += hybrid_loss.item()
            
    return val_loss / len(val_loader)

# --- 5C. MRR CALCULATION (MODIFIED for single model) ---
# This version only takes 3 arguments
def calculate_mrr(translator_model, val_loader, device):
    translator_model.eval()
    all_reciprocal_ranks = []
    with torch.no_grad():
        for text_batch, image_batch in val_loader:
            text_batch = text_batch.to(device)
            image_batch = image_batch.to(device)
            
            pred_embeddings_norm = translator_model(text_batch)
            target_embeddings_norm = F.normalize(image_batch, p=2, dim=1)
            sim_matrix = pred_embeddings_norm @ target_embeddings_norm.T
            
            correct_scores = torch.diag(sim_matrix).unsqueeze(1)
            ranks = (sim_matrix >= correct_scores).sum(dim=1).float()
            reciprocal_ranks = 1.0 / ranks
            all_reciprocal_ranks.append(reciprocal_ranks)
            
    mrr = torch.cat(all_reciprocal_ranks).mean().item()
    return mrr

# --- 5D. MIXUP (Unchanged) ---
def mixup_data(x, y, alpha=0.2):
    if alpha > 0: lam = np.random.beta(alpha, alpha)
    else: lam = 1
    batch_size = x.size(0)
    indices = torch.randperm(batch_size, device=x.device)
    mixed_x = lam * x + (1 - lam) * x[indices, :]
    mixed_y = lam * y + (1 - lam) * y[indices, :]
    return mixed_x, mixed_y

# Main training

In [19]:
# # --- 5. FINAL Main Training Script (MLP-Only + Label Smoothing) ---
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from tqdm import tqdm
# import copy
# import itertools
# import numpy as np
# import json

# # --- Store MRR for each fold ---
# all_fold_mrr = []

# # =====================================================================================
# #  K-FOLD OUTER LOOP
# # =====================================================================================
# for fold, (train_image_indices, val_image_indices) in enumerate(kfold.split(image_indices)):
#     print(f"\n{'='*50}")
#     print(f"--- STARTING FOLD {fold+1}/{CONFIG['N_FOLDS']} ---")
#     print(f"{'='*50}")

#     # --- 1. Create Datasets & DataLoaders for this fold ---
    
#     train_image_emb = all_image_embeddings[train_image_indices]
#     val_image_emb = all_image_embeddings[val_image_indices]

#     train_image_emb_expanded = train_image_emb.repeat_interleave(num_captions_per_image, dim=0)

#     train_caption_indices = [i for idx in train_image_indices for i in range(idx * num_captions_per_image, (idx + 1) * num_captions_per_image)]
#     val_caption_indices = [i for idx in val_image_indices for i in range(idx * num_captions_per_image, (idx + 1) * num_captions_per_image)]

#     train_text_emb = all_text_embeddings[train_caption_indices]
#     val_text_emb = all_text_embeddings[val_caption_indices]
    
#     val_text_emb_one_per_image = val_text_emb[::num_captions_per_image]

#     train_dataset = TensorDataset(train_text_emb, train_image_emb_expanded)
#     val_dataset = TensorDataset(val_text_emb_one_per_image, val_image_emb)

#     # --- Use the seeded DataLoaders ---
#     train_loader = DataLoader(
#         train_dataset, 
#         batch_size=CONFIG['BATCH_SIZE'], 
#         shuffle=True,
#         worker_init_fn=seed_worker, 
#         generator=g                 
#     )
#     val_loader = DataLoader(val_dataset, batch_size=CONFIG['BATCH_SIZE'])

#     print(f"Fold {fold+1}: Train size: {len(train_dataset)}, Val size: {len(val_dataset)}")

#     # --- Save split info ---
#     try:
#         train_ids = all_image_ids[train_image_indices].tolist()
#         val_ids = all_image_ids[val_image_indices].tolist()
#         split_info = {"fold": fold, "train_image_ids": train_ids, "val_image_ids": val_ids}
#         filename = f"fold_{fold}_split_info.json"
#         with open(filename, 'w') as f: json.dump(split_info, f, indent=4)
#         print(f"Saved split info for fold {fold+1} to {filename}")
#     except Exception as e:
#         print(f"Error saving split info for fold {fold+1}: {e}")

#     # --- 2. Re-initialize Models, Losses, and Optimizer ---
    
#     translator_model = TranslatorMLP(
#         in_features=all_text_embeddings.shape[1], 
#         out_features=all_image_embeddings.shape[1], 
#         hidden_features=CONFIG['HIDDEN_FEATURES'],
#         num_blocks=CONFIG['NUM_BLOCKS'],
#         dropout_rate=CONFIG['DROPOUT_RATE']
#     ).to(CONFIG['DEVICE'])

#     criterion_triplet = TripletLoss(margin=CONFIG['MARGIN'])
    
#     # --- MODIFIED: Pass label_smoothing to the loss function ---
#     criterion_pure = PureContrastiveLoss(
#         temperature=CONFIG['TEMPERATURE'],
#         label_smoothing=CONFIG['LABEL_SMOOTHING'] 
#     )

#     optimizer = optim.AdamW(
#         translator_model.parameters(),
#         lr=CONFIG['LR'],
#         weight_decay=CONFIG['WEIGHT_DECAY']
#     )
#     total_steps = len(train_loader) * CONFIG['EPOCHS']
#     scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps)
    
#     # --- 3. Training Loop for this fold ---
#     best_finetune_mrr = -1.0
#     patience_counter = 0

#     for epoch in range(CONFIG['EPOCHS']):
#         translator_model.train()
#         total_train_loss = 0
        
#         batch_iterator = tqdm(train_loader, desc=f"Fold {fold+1} Epoch {epoch+1}/{CONFIG['EPOCHS']}")
        
#         for i, (text_batch, image_batch) in enumerate(batch_iterator):
#             text_batch = text_batch.to(CONFIG['DEVICE'])
#             image_batch = image_batch.to(CONFIG['DEVICE'])
            
#             target_embeddings = F.normalize(image_batch, p=2, dim=1)
#             mixed_text, mixed_targets = mixup_data(text_batch, target_embeddings, alpha=CONFIG['MIXUP_ALPHA'])

#             pred_embeddings = translator_model(mixed_text)
            
#             loss_triplet = criterion_triplet(pred_embeddings, mixed_targets)
#             loss_pure = criterion_pure(pred_embeddings, mixed_targets)
#             loss = (CONFIG['HYBRID_ALPHA'] * loss_triplet) + ((1 - CONFIG['HYBRID_ALPHA']) * loss_pure)
            
#             loss = loss / CONFIG['ACCUMULATION_STEPS']
#             loss.backward()

#             if (i + 1) % CONFIG['ACCUMULATION_STEPS'] == 0:
#                 optimizer.step()
#                 scheduler.step()
#                 optimizer.zero_grad()

#             total_train_loss += loss.item() * CONFIG['ACCUMULATION_STEPS']
#             batch_iterator.set_postfix({"Train Loss": f"{loss.item() * CONFIG['ACCUMULATION_STEPS']:.4f}"})
            
#         # --- Validation ---
#         avg_train_loss = total_train_loss / len(train_loader)
        
#         # --- MODIFIED: Pass correct arguments to validation_fn ---
#         val_loss = validation_fn(
#             translator_model, 
#             val_loader, 
#             criterion_triplet, 
#             criterion_pure, 
#             CONFIG['HYBRID_ALPHA'], 
#             CONFIG['DEVICE']
#         )
#         val_mrr = calculate_mrr(translator_model, val_loader, CONFIG['DEVICE'])
        
#         current_lr = scheduler.get_last_lr()[0]
#         # Added val_loss to the printout
#         print(f"Fold {fold+1} Epoch {epoch+1} -> Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MRR: {val_mrr:.4f} | LR: {current_lr:e}")
        
#         if val_mrr > best_finetune_mrr:
#             best_finetune_mrr = val_mrr
            
#             model_path = CONFIG['MODEL_PATH_TPL'].format(fold)
#             torch.save(translator_model.state_dict(), model_path)
#             print(f"  🏆 New best MRR for Fold {fold+1}! Saving model to {model_path}...")
#             patience_counter = 0
#         else:
#             patience_counter += 1
#             if patience_counter >= CONFIG['PATIENCE']:
#                 print(f"Early stopping triggered for Fold {fold+1}.")
#                 break

#     print(f"\n✅ Fold {fold+1} finished. Best validation MRR: {best_finetune_mrr:.4f}")
#     all_fold_mrr.append(best_finetune_mrr)

# print(f"\n{'='*50}")
# print(f"✅ All {CONFIG['N_FOLDS']} folds trained! ")
# print(f"Mean Validation MRR: {np.mean(all_fold_mrr):.4f} (Std: {np.std(all_fold_mrr):.4f})")
# print(f"{'='*50}")

In [20]:
# --- 16. FINAL Main Training Script (MODIFIED for 1-to-1 Clean Data) ---
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import copy
import itertools
import numpy as np
import json

# --- Store MRR for each fold ---
all_fold_mrr = []

# =====================================================================================
#  K-FOLD OUTER LOOP (Simplified for 1-to-1 data)
# =====================================================================================
# kfold and pair_indices come from Cell 10
for fold, (train_pair_indices, val_pair_indices) in enumerate(kfold.split(pair_indices)):
    print(f"\n{'='*50}")
    print(f"--- STARTING FOLD {fold+1}/{CONFIG['N_FOLDS']} ---")
    print(f"{'='*50}")

    # --- 1. Create Datasets & DataLoaders for this fold ---
    
    # Simple indexing using the pair indices
    train_text_emb = all_text_embeddings[train_pair_indices]
    train_image_emb = all_image_embeddings[train_pair_indices]
    
    val_text_emb = all_text_embeddings[val_pair_indices]
    val_image_emb = all_image_embeddings[val_pair_indices]

    # Create simple 1-to-1 TensorDatasets
    train_dataset = TensorDataset(train_text_emb, train_image_emb)
    val_dataset = TensorDataset(val_text_emb, val_image_emb)

    # --- Use the seeded DataLoaders ---
    train_loader = DataLoader(
        train_dataset, 
        batch_size=CONFIG['BATCH_SIZE'], 
        shuffle=True,
        worker_init_fn=seed_worker, 
        generator=g                 
    )
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['BATCH_SIZE'])

    print(f"Fold {fold+1}: Train size: {len(train_dataset)}, Val size: {len(val_dataset)}")

    # (Optional) We don't need to save split info, as it's just indices

    # --- 2. Re-initialize Models, Losses, and Optimizer ---
    translator_model = TranslatorMLP(
        in_features=all_text_embeddings.shape[1], 
        out_features=all_image_embeddings.shape[1], 
        hidden_features=CONFIG['HIDDEN_FEATURES'],
        num_blocks=CONFIG['NUM_BLOCKS'],
        dropout_rate=CONFIG['DROPOUT_RATE']
    ).to(CONFIG['DEVICE'])

    criterion_triplet = TripletLoss(margin=CONFIG['MARGIN'])
    criterion_pure = PureContrastiveLoss(
        temperature=CONFIG['TEMPERATURE'],
        label_smoothing=CONFIG['LABEL_SMOOTHING'] 
    )

    # --- Use Optimizer from CONFIG (e.g., AdamW) ---
    optimizer = optim.AdamW(
        translator_model.parameters(),
        lr=CONFIG['LR'],
        weight_decay=CONFIG['WEIGHT_DECAY']
    )
    
    total_steps = len(train_loader) * CONFIG['EPOCHS'] // CONFIG['ACCUMULATION_STEPS']
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps)
    
    # --- 3. Training Loop for this fold ---
    best_finetune_mrr = -1.0
    patience_counter = 0

    for epoch in range(CONFIG['EPOCHS']):
        translator_model.train()
        total_train_loss = 0
        
        batch_iterator = tqdm(train_loader, desc=f"Fold {fold+1} Epoch {epoch+1}/{CONFIG['EPOCHS']}")
        
        # --- MODIFIED: Loop is now simpler (text, image) ---
        for i, (text_batch, image_batch) in enumerate(batch_iterator):
            text_batch = text_batch.to(CONFIG['DEVICE'])
            image_batch = image_batch.to(CONFIG['DEVICE'])
            
            target_embeddings = F.normalize(image_batch, p=2, dim=1)
            mixed_text, mixed_targets = mixup_data(text_batch, target_embeddings, alpha=CONFIG['MIXUP_ALPHA'])

            pred_embeddings = translator_model(mixed_text)
            
            loss_triplet = criterion_triplet(pred_embeddings, mixed_targets)
            loss_pure = criterion_pure(pred_embeddings, mixed_targets)
            loss = (CONFIG['HYBRID_ALPHA'] * loss_triplet) + ((1 - CONFIG['HYBRID_ALPHA']) * loss_pure)
            
            loss = loss / CONFIG['ACCUMULATION_STEPS']
            loss.backward()

            if (i + 1) % CONFIG['ACCUMULATION_STEPS'] == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

            total_train_loss += loss.item() * CONFIG['ACCUMULATION_STEPS']
            batch_iterator.set_postfix({"Train Loss": f"{loss.item() * CONFIG['ACCUMULATION_STEPS']:.4f}"})
            
        # --- Validation ---
        avg_train_loss = total_train_loss / len(train_loader)
        
        val_loss = validation_fn(
            translator_model, 
            val_loader, 
            criterion_triplet, 
            criterion_pure, 
            CONFIG['HYBRID_ALPHA'], 
            CONFIG['DEVICE']
        )
        val_mrr = calculate_mrr(translator_model, val_loader, CONFIG['DEVICE'])
        
        current_lr = scheduler.get_last_lr()[0]
        print(f"Fold {fold+1} Epoch {epoch+1} -> Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MRR: {val_mrr:.4f} | LR: {current_lr:e}")
        
        if val_mrr > best_finetune_mrr:
            best_finetune_mrr = val_mrr
            
            model_path = CONFIG['MODEL_PATH_TPL'].format(fold)
            torch.save(translator_model.state_dict(), model_path)
            print(f"  🏆 New best MRR for Fold {fold+1}! Saving model to {model_path}...")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= CONFIG['PATIENCE']:
                print(f"Early stopping triggered for Fold {fold+1}.")
                break

    print(f"\n✅ Fold {fold+1} finished. Best validation MRR: {best_finetune_mrr:.4f}")
    all_fold_mrr.append(best_finetune_mrr)

print(f"\n{'='*50}")
print(f"✅ All {CONFIG['N_FOLDS']} folds trained! ")
print(f"Mean Validation MRR: {np.mean(all_fold_mrr):.4f} (Std: {np.std(all_fold_mrr):.4f})")
print(f"{'='*50}")


--- STARTING FOLD 1/10 ---
Fold 1: Train size: 111114, Val size: 12346


Fold 1 Epoch 1/100: 100%|██████████| 435/435 [00:09<00:00, 45.00it/s, Train Loss=1.0347]


Fold 1 Epoch 1 -> Train Loss: 4.3901 | Val Loss: 3.7042 | Val MRR: 0.4624 | LR: 1.774487e-05
  🏆 New best MRR for Fold 1! Saving model to best_translator_mlp_fold_0.pth...


Fold 1 Epoch 2/100: 100%|██████████| 435/435 [00:08<00:00, 48.78it/s, Train Loss=1.4847]


Fold 1 Epoch 2 -> Train Loss: 3.3210 | Val Loss: 3.2806 | Val MRR: 0.5400 | LR: 1.773179e-05
  🏆 New best MRR for Fold 1! Saving model to best_translator_mlp_fold_0.pth...


...

Fold 10 Epoch 98 -> Train Loss: 1.5150 | Val Loss: 1.9397 | Val MRR: 0.7446 | LR: 2.167776e-08


Fold 10 Epoch 99/100: 100%|██████████| 435/435 [00:08<00:00, 49.68it/s, Train Loss=1.0126]


Fold 10 Epoch 99 -> Train Loss: 1.5064 | Val Loss: 1.9410 | Val MRR: 0.7445 | LR: 6.598866e-09


Fold 10 Epoch 100/100: 100%|██████████| 435/435 [00:08<00:00, 49.56it/s, Train Loss=0.8942]


Fold 10 Epoch 100 -> Train Loss: 1.5036 | Val Loss: 1.9381 | Val MRR: 0.7443 | LR: 2.314402e-10

✅ Fold 10 finished. Best validation MRR: 0.7451

✅ All 10 folds trained! 
Mean Validation MRR: 0.7449 (Std: 0.0040)


# Metrics

 # Submission

In [21]:
# # --- 8. GENERATE FINAL SUBMISSION FILE (for VAE -> MLP) ---
# import numpy as np
# import torch
# from torch.utils.data import DataLoader
# import pandas as pd
# import json
# from tqdm import tqdm

# print("--- Generating submission.csv file for Kaggle (VAE -> MLP) ---")

# # --- Load the test data ---
# try:
#     test_data = np.load("/kaggle/input/test.clean.npz", allow_pickle=True)
#     test_text_emb = torch.from_numpy(test_data['captions/embeddings']).float()
#     test_caption_ids = test_data['captions/ids']
#     print("Test data and caption IDs loaded successfully.")
# except FileNotFoundError:
#     print("ERROR: Please make sure the 'test.clean.npz' file is in the correct directory.")

# # --- Load your best VAE and Translator models ---
# text_ve = TextVariationalEncoder(
#     in_features=all_text_embeddings.shape[1],
#     hidden_features=CONFIG['LATENT_DIM'],
#     latent_dim=CONFIG['LATENT_DIM']
# ).to(CONFIG['DEVICE'])

# translator_model = TranslatorMLP(
#     in_features=CONFIG['LATENT_DIM'],
#     out_features=all_image_embeddings.shape[1], 
#     hidden_features=CONFIG['HIDDEN_FEATURES'],
#     num_blocks=CONFIG['NUM_BLOCKS'],
#     dropout_rate=CONFIG['DROPOUT_RATE']
# ).to(CONFIG['DEVICE'])

# text_ve.load_state_dict(torch.load(CONFIG['TEXT_VE_PATH']))
# translator_model.load_state_dict(torch.load(CONFIG['TRANSLATOR_PATH']))
# text_ve.eval()
# translator_model.eval()

# # --- Create predictions in batches ---
# predictions = []
# test_loader = DataLoader(test_text_emb, batch_size=CONFIG['BATCH_SIZE'])

# with torch.no_grad():
#     for text_batch in tqdm(test_loader, desc="Generating Predictions"):
#         text_batch = text_batch.to(CONFIG['DEVICE'])
        
#         #  novedad -> VAE -> MLP flow
#         # We sample (z) just like in training
#         z, _, _ = text_ve(text_batch)
#         pred_batch = translator_model(z)
        
#         predictions.append(pred_batch.cpu().numpy())

# final_predictions = np.concatenate(predictions, axis=0)

# # --- Format for CSV submission ---
# embedding_json_list = [json.dumps(embedding.tolist()) for embedding in final_predictions]

# submission_df = pd.DataFrame({
#     'id': test_caption_ids,
#     'embedding': embedding_json_list
# })

# # --- Save the DataFrame to a CSV file ---
# submission_df.to_csv("submission.csv", index=False)

# print(f"\n✅ Submission file 'submission.csv' has been generated successfully!")
# print("Here's a preview of the first 5 rows:")
# print(submission_df.head())

In [22]:
# --- 8. GENERATE FINAL SUBMISSION FILE (for MLP-Only) ---
import numpy as np
import torch
from torch.utils.data import DataLoader
import pandas as pd
import json
from tqdm import tqdm

print("--- Generating submission.csv using K-Fold MLP models (Averaging) ---")

# --- 1. Load the test data ---
try:
    test_data = np.load("/kaggle/input/test.clean.npz", allow_pickle=True)
    test_text_emb = torch.from_numpy(test_data['captions/embeddings']).float()
    test_caption_ids = test_data['captions/ids']
    test_loader = DataLoader(test_text_emb, batch_size=CONFIG['BATCH_SIZE'])
    print(f"Test data and caption IDs loaded: {len(test_caption_ids)} samples.")
except FileNotFoundError:
    print("ERROR: Please make sure the 'test.clean.npz' file is in the correct directory.")

# --- 2. Loop through all folds to get predictions ---
all_predictions_list = []

for fold in range(CONFIG['N_FOLDS']):
    print(f"\n--- Generating predictions for Fold {fold+1}/{CONFIG['N_FOLDS']} ---")
    
    # --- Define model path for this fold ---
    model_path = CONFIG['MODEL_PATH_TPL'].format(fold)

    # --- MODIFIED: Load only the TranslatorMLP ---
    translator_model = TranslatorMLP(
        in_features=all_text_embeddings.shape[1], # 1024
        out_features=all_image_embeddings.shape[1], # 1536
        hidden_features=CONFIG['HIDDEN_FEATURES'],
        num_blocks=CONFIG['NUM_BLOCKS'],
        dropout_rate=CONFIG['DROPOUT_RATE']
    ).to(CONFIG['DEVICE'])

    try:
        translator_model.load_state_dict(torch.load(model_path))
    except FileNotFoundError:
        print(f"ERROR: Could not find model file for fold {fold}: {model_path}")
        continue 
        
    translator_model.eval()

    # --- Create predictions in batches for this fold ---
    fold_predictions = []
    with torch.no_grad():
        for text_batch in tqdm(test_loader, desc=f"Predicting Fold {fold+1}"):
            text_batch = text_batch.to(CONFIG['DEVICE'])
            
            # --- MODIFIED: Direct MLP flow ---
            pred_batch = translator_model(text_batch)
            
            fold_predictions.append(pred_batch.cpu().numpy())
    
    all_predictions_list.append(np.concatenate(fold_predictions, axis=0))

if not all_predictions_list:
    print("\nNo predictions were generated. Cannot create submission file.")
else:
    # --- 3. Average the predictions from all folds ---
    print(f"\n--- Averaging predictions from {len(all_predictions_list)} folds ---")
    avg_predictions = np.mean(all_predictions_list, axis=0)
    
    # --- 4. Format for CSV submission ---
    embedding_json_list = [json.dumps(embedding.tolist()) for embedding in avg_predictions]

    submission_df = pd.DataFrame({
        'id': test_caption_ids,
        'embedding': embedding_json_list
    })

    # --- 5. Save the DataFrame to a CSV file ---
    submission_df.to_csv("submission.csv", index=False)

    print(f"\n✅ Ensembled submission file 'submission.csv' has been generated successfully!")
    print("Here's a preview of the first 5 rows:")
    print(submission_df.head())

--- Generating submission.csv using K-Fold MLP models (Averaging) ---
ERROR: Please make sure the 'test.clean.npz' file is in the correct directory.

--- Generating predictions for Fold 1/10 ---


NameError: name 'test_loader' is not defined

# OPTUNA

In [ ]:
# --- 6. Optuna Objective Function (CORRECTED for VAE -> MLP) ---
import optuna
from tqdm import tqdm
import copy
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# --- 
# NEW: Copied helper functions from cell 15 to ensure Optuna uses the correct versions
# ---

def validation_fn_optuna(text_ve, translator_model, val_loader, criterion_triplet, criterion_pure, alpha, kld_weight, device):
    """Helper for Optuna objective"""
    text_ve.eval()
    translator_model.eval()
    val_loss = 0
    with torch.no_grad():
        for text_batch, image_batch in val_loader:
            text_batch, image_batch = text_batch.to(device), image_batch.to(device)
            
            z, mu, log_var = text_ve(text_batch)
            pred_embeddings = translator_model(z)
            target_embeddings = F.normalize(image_batch, p=2, dim=1)
            
            loss_triplet = criterion_triplet(pred_embeddings, target_embeddings)
            loss_pure = criterion_pure(pred_embeddings, target_embeddings)
            hybrid_loss = (alpha * loss_triplet) + ((1 - alpha) * loss_pure)
            
            kld_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            kld_loss = kld_loss / text_batch.size(0)
            
            loss = hybrid_loss + (kld_weight * kld_loss)
            val_loss += loss.item()
            
    return val_loss / len(val_loader)

def calculate_mrr_optuna(text_ve, translator_model, val_loader, device):
    """Helper for Optuna objective"""
    text_ve.eval()
    translator_model.eval()
    all_reciprocal_ranks = []
    with torch.no_grad():
        for text_batch, image_batch in val_loader:
            text_batch = text_batch.to(device)
            image_batch = image_batch.to(device)
            
            z, _, _ = text_ve(text_batch) # We sample z
            pred_embeddings_norm = translator_model(z)
            
            target_embeddings_norm = F.normalize(image_batch, p=2, dim=1)
            sim_matrix = pred_embeddings_norm @ target_embeddings_norm.T
            
            correct_scores = torch.diag(sim_matrix).unsqueeze(1)
            ranks = (sim_matrix >= correct_scores).sum(dim=1).float()
            reciprocal_ranks = 1.0 / ranks
            all_reciprocal_ranks.append(reciprocal_ranks)
            
    mrr = torch.cat(all_reciprocal_ranks).mean().item()
    return mrr

def mixup_data_optuna(x, y, alpha=0.2):
    """Helper for Optuna objective"""
    if alpha > 0: lam = np.random.beta(alpha, alpha)
    else: lam = 1
    batch_size = x.size(0)
    indices = torch.randperm(batch_size, device=x.device)
    mixed_x = lam * x + (1 - lam) * x[indices, :]
    mixed_y = lam * y + (1 - lam) * y[indices, :]
    return mixed_x, mixed_y

# ---
# NEW: Corrected Objective Function
# ---

def objective(trial):
    """
    Corrected objective to tune the two-model (TextVE -> TranslatorMLP) architecture.
    """
    
    # --- 1. Define Hyperparameter Search Space ---
    params = {
        'LR': trial.suggest_float('LR', 1e-6, 5e-4, log=True),
        'HYBRID_ALPHA': trial.suggest_float('HYBRID_ALPHA', 0.0, 1.0), # Tuned 0.0-1.0
        'MARGIN': trial.suggest_float('MARGIN', 0.1, 0.5),
        'TEMPERATURE': trial.suggest_float('TEMPERATURE', 0.01, 0.1, log=True),
        
        # --- VAE-specific params ---
        'LATENT_DIM': trial.suggest_categorical('LATENT_DIM', [1024, 1536, 2048, 3072]),
        'KLD_WEIGHT': trial.suggest_float('KLD_WEIGHT', 1e-7, 1e-4, log=True),
        
        # --- Regularizers ---
        'WEIGHT_DECAY': trial.suggest_float('WEIGHT_DECAY', 1e-5, 1e-2, log=True),
        'DROPOUT_RATE': trial.suggest_float('DROPOUT_RATE', 0.1, 0.5),
        'MIXUP_ALPHA': trial.suggest_float('MIXUP_ALPHA', 0.0, 0.4), # Start from 0
        
        # --- Architecture ---
        'BATCH_SIZE' : trial.suggest_categorical('BATCH_SIZE', [256, 512]),
        'NUM_BLOCKS': trial.suggest_int('NUM_BLOCKS', 2, 6), 
        'HIDDEN_FEATURES': trial.suggest_categorical('HIDDEN_FEATURES', [2048, 3072, 4096])
    }
    
    # --- Config for this trial ---
    EPOCHS = 50   
    PATIENCE = 10           
    ACCUMULATION_STEPS = 2 # Hardcode from main config
    DEVICE = CONFIG['DEVICE'] # Use global config device

    # --- Create DataLoaders (using global train_dataset and val_dataset) ---
    try:
        train_loader = DataLoader(train_dataset, batch_size=params['BATCH_SIZE'], shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=params['BATCH_SIZE'])
    except NameError: 
        print("Error: train_dataset or val_dataset not defined. Run K-Fold loop cell first.")
        return -1.0 

    # --- 2. Instantiate the TWO-MODEL setup ---
    text_ve = TextVariationalEncoder(
        in_features=all_text_embeddings.shape[1],
        hidden_features=params['LATENT_DIM'], # Tuned
        latent_dim=params['LATENT_DIM']       # Tuned
    ).to(DEVICE)

    translator_model = TranslatorMLP(
        in_features=params['LATENT_DIM'],     # Tuned
        out_features=all_image_embeddings.shape[1], 
        hidden_features=params['HIDDEN_FEATURES'], # Tuned
        num_blocks=params['NUM_BLOCKS'],           # Tuned
        dropout_rate=params['DROPOUT_RATE']        # Tuned
    ).to(DEVICE)

    # --- Create *both* loss functions ---
    criterion_triplet = TripletLoss(margin=params['MARGIN'])
    criterion_pure = PureContrastiveLoss(temperature=params['TEMPERATURE'])
    
    # --- Optimizer for BOTH models ---
    optimizer = optim.AdamW(
        itertools.chain(text_ve.parameters(), translator_model.parameters()),
        lr=params['LR'],
        weight_decay=params['WEIGHT_DECAY']
    )
    total_steps = len(train_loader) * EPOCHS
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps)
    
    best_val_mrr = -1.0 # Track MRR now
    best_text_ve_state = None
    best_translator_state = None
    patience_counter = 0

    for epoch in range(EPOCHS):
        text_ve.train()
        translator_model.train()
        
        for i, (text_batch, image_batch) in enumerate(train_loader):
            text_batch = text_batch.to(DEVICE)
            image_batch = image_batch.to(DEVICE)
            
            target_embeddings = F.normalize(image_batch, p=2, dim=1)
            
            # Apply Mixup
            mixed_text, mixed_targets = mixup_data_optuna(text_batch, target_embeddings, alpha=params['MIXUP_ALPHA'])
            
            # --- Correct two-model forward pass ---
            z, mu, log_var = text_ve(mixed_text)
            pred_embeddings = translator_model(z)
            
            # Calculate the Hybrid Loss
            loss_triplet = criterion_triplet(pred_embeddings, mixed_targets)
            loss_pure = criterion_pure(pred_embeddings, mixed_targets)
            hybrid_loss = (params['HYBRID_ALPHA'] * loss_triplet) + ((1 - params['HYBRID_ALPHA']) * loss_pure)
            
            # Add the KLD Loss
            kld_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            kld_loss = kld_loss / mixed_text.size(0) # Average over batch
            
            loss = hybrid_loss + (params['KLD_WEIGHT'] * kld_loss)
            
            loss = loss / ACCUMULATION_STEPS
            loss.backward()

            if (i + 1) % ACCUMULATION_STEPS == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
        
        # --- Validation ---
        # Use the correct MRR calculation function
        val_mrr = calculate_mrr_optuna(text_ve, translator_model, val_loader, DEVICE)
        
        if val_mrr > best_val_mrr:
            best_val_mrr = val_mrr
            best_text_ve_state = copy.deepcopy(text_ve.state_dict())
            best_translator_state = copy.deepcopy(translator_model.state_dict())
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                # print(f"Optuna trial {trial.number} early stopping at epoch {epoch}")
                break 
    
    # --- 3. Return the 1-vs-batch MRR score ---
    # No need to load best state, it's already in best_val_mrr
    return best_val_mrr

In [ ]:
# # Create a new study
# study = optuna.create_study(direction="maximize")

# # Run for 50 trials to explore this new space
# print("--- Starting Final Optuna Search for combined loss ---")
# study.optimize(objective, n_trials=60) 

# # --- Get Your Best Parameters! ---
# print("\n🎉 Final Optimization Finished! 🎉")
# trial = study.best_trial
# print(f"  Best MRR: {trial.value:.4f}")
# print("  Best Parameters (for HybridMLP): ")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")